# 느리게 갱신되는 세그먼트 트리 (Segment Tree With Lazy Propagation)

`-` 참고: https://book.acmicpc.net/ds/segment-tree-lazy-propagation

## 구간 합 구하기 2

- 문제 출처: [백준 10999번](https://www.acmicpc.net/problem/10999)

`-` 느리게 갱신되는 세그먼트 트리 기본 문제이다

`-` 물론 나는 이걸 어떻게 구현하는지 모른다

`-` 단순히 구간 업데이트를 개별로 점 업데이트 하는 건 시간 초과이다

`-` 실제로 출력을 할 때만 값을 알면 되니 그 전까지 구간 업데이트를 모아둔 뒤 출력 쿼리 때 계산하자

`-` 이를 위해 임의의 노드에 대해 구간 업데이트로 해당 구간에 누적된 값을 나타내는 lazy를 도입하자

`-` 구간 업데이트에서 해당 구간을 담당하는 노드의 lazy 값을 갱신하자

`-` 예컨대 전체 구간에 $d$를 더한다면 루트 노드의 lazy 값에 $d$를 더해주면 된다

`-` 그런데 단순히 lazy 값만 갱신하면 문제가 생긴다

`-` 예컨대 루트 노드의 오른쪽 자식이 담당하는 구간에 $d$를 더한다고 해보자

`-` 그럼 $\operatorname{lazy}[3] = \operatorname{lazy}[3] + d$가 된다

`-` 이때 전체 구간의 합을 계산해보자

`-` 이는 $\operatorname{tree}[1]$인데 $\operatorname{tree}[1]$은 이전과 변화가 없으니 틀린 결과를 도출한다

`-` 임의의 노드가 담당하는 구간을 $(s,e)$라 하고 갱신할 구간을 $(l,r)$이라 할 때 $l \le s \le e \le r$이면 해당 노드의 lazy 값에 반영하면 된다

`-` 그렇지 않고 일부 구간만 겹칠 땐 tree 배열에 겹친 부분의 구간 합을 누적한 뒤 자식 노드에 $d$만큼 구간 업데이트 된다는 걸 전파하자

`-` 이제 구간 합을 계산하는 쿼리를 처리해야 한다

`-` 기본적으로 $l \le s \le e \le r$이면 해당 노드가 저장하고 있는 구간 합에 lazy 값과 구간의 길이를 곱한 값을 더해서 반환하면 된다

`-` 이제 일부 구간이 겹친다고 해보자

`-` 나는 [수열과 쿼리 21](https://www.acmicpc.net/problem/16975) 문제에서 해결한 것처럼 $\operatorname{aggregate}$ 변수를 도입했다

`-` 그 후 자식 노드를 기준으로 겹친 구간만큼 lazy 값을 계산해 $\operatorname{aggregate}$에 누적하여 전파했다

`-` 그런데 해당 부분을 계속 틀려서 chatgpt의 도움을 조금 받았다

`-` 구간 합 쿼리를 실행했으므로 여태까지 누적된 lazy 값을 tree에 반영해주자

`-` 그리고 lazy 값을 자식들에게 전파한 뒤 자기 자신의 lazy 값을 $0$으로 초기화하면 된다

`-` 구간 합이므로 자식들의 쿼리 결과의 합을 반환하면 마무리된다

`-` 느리게 갱신되는 세그먼트 트리의 시간 복잡도는 일반적인 세그먼트 트리의 시간 복잡도와 동일하다

`-` 따라서 전체 알고리즘의 시간 복잡도는 $O((M+K)\log N)$이다

In [47]:
import math


def init_tree(array, tree, node, start, end):
    if start == end:
        tree[node] = array[start]
        return
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    init_tree(array, tree, left_child, start, mid)
    init_tree(array, tree, right_child, mid + 1, end)
    tree[node] = tree[left_child] + tree[right_child]


def update_tree(left, right, value, tree, lazy, node, start, end):
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        lazy[node] += value
        return value * (end - start + 1)
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    left_result = update_tree(left, right, value, tree, lazy, left_child, start, mid)
    right_result = update_tree(left, right, value, tree, lazy, right_child, mid + 1, end)
    tree[node] += left_result + right_result
    return left_result + right_result


def range_sum_query(left, right, tree, lazy, node, start, end):
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        return tree[node] + lazy[node] * (end - start + 1)
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    tree[node] += lazy[node] * (end - start + 1)
    lazy[left_child] += lazy[node]
    lazy[right_child] += lazy[node]
    lazy[node] = 0
    left_sum = range_sum_query(left, right, tree, lazy, left_child, start, mid)
    right_sum = range_sum_query(left, right, tree, lazy, right_child, mid + 1, end)
    return left_sum + right_sum


def solution():
    N, M, K = map(int, input().split())
    array = [int(input()) for _ in range(N)]
    h = math.ceil(math.log2(N))
    tree_size = 2**(h + 1) - 1
    tree = [0] * (tree_size + 1)
    lazy = [0] * (tree_size + 1)
    start, end = 0, N - 1
    root = 1
    init_tree(array, tree, root, start, end)
    for _ in range(M + K):
        query = list(map(int, input().split()))
        a = query[0]
        if a == 1:
            b, c, d = query[1:]
            update_tree(b - 1, c - 1, d, tree, lazy, root, start, end)
        else:
            b, c = query[1:]
            range_sum = range_sum_query(b - 1, c - 1, tree, lazy, root, start, end)
            print(range_sum)


solution()

# input
# 5 2 2
# 1
# 2
# 3
# 4
# 5
# 1 3 4 6
# 2 2 5
# 1 1 3 -2
# 2 2 5

 5 2 2
 1
 2
 3
 4
 5
 1 3 4 6
 2 2 5


26


 1 1 3 -2
 2 2 5


22


`-` 문제 풀었으니까 제대로 공부하고 오자

`-` 보통은 노드의 lazy 값을 먼저 갱신한 후 구간 업데이트 또는 구간 합 쿼리를 수행한다

`-` 근데 나는 구간 합 쿼리를 수행할 때만 lazy 값을 갱신했다

`-` 원래 세그먼트 트리에서 갱신할 땐 `tree[node] = tree[left_child] + tree[right_child]`와 같이 수행했다

`-` 근데 이렇게 할려면 미뤄둔 lazy 값을 tree에 먼저 반영을 해야 한다

`-` 쿼리 구간을 벗어나는 노드에 대해선 아무 작업도 수행하지 않는데 lazy 값이 있어도 무시하므로 트리 갱신 때 값이 틀려진다

`-` 근데 나는 명시적으로 구간 업데이트로 변화된 값을 반환하도록 했다

`-` 자식들의 결괏값을 `tree[node]`에 누적하므로 정상적으로 동작했다

## XOR

- 문제 출처: [백준 12844번](https://www.acmicpc.net/problem/12844)

`-` 구간 업데이트와 구간 쿼리를 수행하면 된다

`-` 느리게 갱신되는 세그먼트 트리를 배운 나는 무적이다

`-` xor 연산은 결합법칙이 성립하며 $k \oplus k = 0$이다

`-` 따라서 tree에 lazy 값을 반영할 때 담당 구간의 길이가 짝수면 $0$만 xor하는 것이니 기존과 같고 홀수면 $\operatorname{lazy}[\operatorname{node}]$와 xor하면 된다

`-` 또한 xor 연산의 항등원은 $0$이니 구간 쿼리에서 범위를 벗어난 경우엔 $0$을 반환하도록 했다

`-` 나머지는 기본적인 느리게 갱신되는 세그먼트 트리와 동일하다

In [24]:
import math


def init_tree(array, tree, node, start, end):
    if start == end:
        tree[node] = array[start]
        return
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    init_tree(array, tree, left_child, start, mid)
    init_tree(array, tree, right_child, mid + 1, end)
    tree[node] = tree[left_child] ^ tree[right_child]


def update_lazy(tree, lazy, node, start, end):
    if lazy[node] == 0:
        return
    if (end - start + 1) % 2 == 1:
        tree[node] ^= lazy[node]
    if start != end:
        lazy[2 * node] ^= lazy[node]
        lazy[2 * node + 1] ^= lazy[node]
    lazy[node] = 0


def update_tree(left, right, value, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if left > end or right < start:
        return
    if left <= start and end <= right:
        lazy[node] ^= value
        update_lazy(tree, lazy, node, start, end)
        return
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    update_tree(left, right, value, tree, lazy, left_child, start, mid)
    update_tree(left, right, value, tree, lazy, right_child, mid + 1, end)
    tree[node] = tree[left_child] ^ tree[right_child]


def range_xor_query(left, right, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        return tree[node]
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    left_xor = range_xor_query(left, right, tree, lazy, left_child, start, mid)
    right_xor = range_xor_query(left, right, tree, lazy, right_child, mid + 1, end)
    return left_xor ^ right_xor


def solution():
    N = int(input())
    array = list(map(int, input().split()))
    M = int(input())
    h = math.ceil(math.log2(N))
    tree_size = 2**(h + 1)
    tree = [0] * tree_size
    lazy = [0] * tree_size
    start, end = 0, N - 1
    root = 1
    init_tree(array, tree, root, start, end)
    answers = []
    for _ in range(M):
        query = list(map(int, input().split()))
        operator = query[0]
        if operator == 1:
            i, j, k = query[1:]
            update_tree(i, j, k, tree, lazy, root, start, end)
        else:
            i, j = query[1:]
            range_xor = range_xor_query(i, j, tree, lazy, root, start, end)
            answers.append(range_xor)
    print("\n".join(map(str, answers)))


solution()

# input
# 5
# 1 2 3 4 5
# 3
# 2 0 4
# 1 2 4 9
# 2 0 4

 5
 1 2 3 4 5
 3
 2 0 4
 1 2 4 9
 2 0 4


1
8


## 스위치

- 문제 출처: [백준 1395번](https://www.acmicpc.net/problem/1395)

`-` 세그먼트 트리의 노드에 구간에 속한 스위치 중 켜진 스위치의 개수를 저장하자

`-` 임의의 노드가 저장하고 있는 구간이 $(s,e)$이고 저장된 값이 $x$라고 하자

`-` 그럼 해당 구간에 속한 스위치의 상태를 반전시키는 것은 노드의 값으로 $x$ 대신 $e- s + 1 - x$를 할당하는 것과 같다

`-` 노드에 $e-s+1-2x$를 더하는 것이라고 생각해도 된다

`-` 그런데 이렇게 하면 노드마다 더하는 값이 다르기에 lazy를 어떻게 설정하지 의문이다

`-` 스위치의 상태를 반전시키는 것에 착안하여 xor 연산을 사용하자

`-` lazy가 $0$ 또는 $1$을 가지도록 만들자

`-` 이는 해당 구간에 속한 스위치의 반전 여부를 나타낸다

`-` 구간 업데이트를 해당 구간에 속한 원소 각각에 대해 $1$과 xor 하는 것이라 생각하면 된다

`-` lazy를 갱신할 때 값이 $1$이라면 노드의 값으로 $x$ 대신 $e- s + 1 - x$를 할당하고 자식에게 lazy를 전파하자

`-` 즉, 구간 업데이트는 $1$과의 xor이며 구간 쿼리는 구간 합을 계산하면 된다

In [2]:
import math


def update_lazy(tree, lazy, node, start, end):
    if lazy[node] == 0:
        return
    tree[node] = end - start + 1 - tree[node]
    if start != end:
        lazy[2 * node] ^= lazy[node]
        lazy[2 * node + 1] ^= lazy[node]
    lazy[node] = 0


def update_tree(left, right, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if left > end or right < start:
        return
    if left <= start and end <= right:
        lazy[node] ^= 1
        update_lazy(tree, lazy, node, start, end)
        return
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    update_tree(left, right, tree, lazy, left_child, start, mid)
    update_tree(left, right, tree, lazy, right_child, mid + 1, end)
    tree[node] = tree[left_child] + tree[right_child]


def range_sum_query(left, right, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        return tree[node]
    mid = (start + end) // 2
    left_child = 2 * node
    right_child = 2 * node + 1
    left_sum = range_sum_query(left, right, tree, lazy, left_child, start, mid)
    right_sum = range_sum_query(left, right, tree, lazy, right_child, mid + 1, end)
    return left_sum + right_sum


def solution():
    N, M = map(int, input().split())
    h = math.ceil(math.log2(N))
    tree_size = 2**(h + 1)
    tree = [0] * tree_size
    lazy = [0] * tree_size
    start, end = 0, N - 1
    root = 1
    answers = []
    for _ in range(M):
        O, S, T = map(int, input().split())
        if O == 0:
            update_tree(S - 1, T - 1, tree, lazy, root, start, end)
        else:
            range_sum = range_sum_query(S - 1, T - 1, tree, lazy, root, start, end)
            answers.append(range_sum)
    print("\n".join(map(str, answers)))


solution()

# input
# 4 5
# 0 1 2
# 0 2 4
# 1 2 3
# 0 2 4
# 1 1 4

 4 5
 0 1 2
 0 2 4
 1 2 3
 0 2 4
 1 1 4


1
2


## 회사 문화 5

- 문제 출처: [백준 18437번](https://www.acmicpc.net/problem/18437)

`-` [스위치](https://www.acmicpc.net/problem/1395) 문제와 [회사 문화 2](https://www.acmicpc.net/problem/14268) 문제의 결합이다

`-` 느리게 갱신되는 세그먼트 트리와 오일러 투어 테크닉을 제대로 구현하면 풀 수 있다

`-` 참고로 구간 쿼리에서 상사 자기 자신은 구간에 포함되지 않으니 주의하자

`-` 일반적인 쿼리 구간에서 왼쪽 경계의 값을 $1$ 증가시키면 된다

`-` DFS에서 진입 순번에만 $1$을 더한다고 생각해도 좋다

`-` 구간 업데이트를 보면 서브 트리에 속한 모든 직원이 컴퓨터를 켜거나 끈다

`-` 따라서 마지막 연산만이 중요하다 (앞서 컴퓨터를 $100$번 꺼도 마지막에 $1$번 켜면 전부 켜져있다)

`-` 컴퓨터를 켜는 건 lazy 값을 $1$로 만들고 컴퓨터를 끄는 건 lazy 값을 $-1$로 만들자

`-` 또한 부모 노드의 lazy 값을 전파할 때 자식 노드의 lazy 값을 부모 노드의 lazy 값으로 설정하면 된다

`-` 한편, 임의의 노드가 담당하는 구간의 길이가 $n$일 때 lazy 값이 $1$이면 컴퓨터를 모두 켜야하니 노드가 가리키는 값 또한 $n$이다

`-` 만약 lazy 값이 $-1$이면 컴퓨터를 모두 꺼야하니 노드가 가리키는 값은 $0$이 된다

`-` 이러한 특성으로 인해 처음에 트리를 초기화하는 대신 루트 노드에 컴퓨터를 켜는 쿼리를 실행하는 걸로 대체했다

`-` 제출하니 시간 초과 발생해서 틀렸다

`-` 리프 노드의 경우 쿼리 구간에서 $L > R$일 수 있다

`-` 이 경우 어차피 해당 직원을 상사로 둔 직원이 없으니 무시하자

`-` 그래도 시간 초과가 발생해서 코드를 꼼꼼히 확인했다

`-` 내가 느리게 갱신되는 세그먼트 트리를 잘못 구현했다;;

`-` 이게 기본 세그먼트 트리를 복붙하고 수정하는 방식으로 진행했는데 구간 쿼리 부분을 제대로 수정하지 않았다

`-` 구간을 노드가 포함하면 더 이상 탐색하지 않아야 하는데 리프 노드까지 탐색을 진행했다

`-` 그래서 시간 초과가 발생했음

`-` 부모 노드의 lazy 값은 $0$이어도 자식 노드의 lazy 값은 $0$이 아닐 수 있으니 갱신을 제대로 하자

`-` 또한 루트 노드의 부모 노드는 $0$으로 주어졌으니 변경하면 안 된다

`-` 즉, 그래프에는 $0$으로 입력하고 dfs 함수에선 다르게 전달하면 안 된다

In [63]:
import math
import sys

sys.setrecursionlimit(10**5 + 2)


def dfs(graph, node, parent, query_ranges):
    global NTH
    query_ranges[node][START] = NTH + 1
    for child in graph[node]:
        if child == parent:
            continue
        NTH += 1
        dfs(graph, child, node, query_ranges)
    query_ranges[node][END] = NTH


def update_lazy(tree, lazy, node, start, end):
    if lazy[node] == 0:
        return
    if lazy[node] == ON:
        tree[node] = end - start + 1
    else:
        tree[node] = 0
    if start != end:
        lazy[2 * node] = lazy[node]
        lazy[2 * node + 1] = lazy[node]
    lazy[node] = 0


def update_range(left, right, state, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if left > end or right < start:
        return
    if left <= start and end <= right:
        lazy[node] = state
        update_lazy(tree, lazy, node, start, end)
        return
    mid = (start + end) // 2
    update_range(left, right, state, tree, lazy, 2 * node, start, mid)
    update_range(left, right, state, tree, lazy, 2 * node + 1, mid + 1, end)
    tree[node] = tree[2 * node] + tree[2 * node + 1]


def range_query(left, right, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        return tree[node]
    mid = (start + end) // 2
    left_count = range_query(left, right, tree, lazy, 2 * node, start, mid)
    right_count = range_query(left, right, tree, lazy, 2 * node + 1, mid + 1, end)
    return left_count + right_count


def solution():
    global START, END, NTH, ON, OFF
    N = int(input())
    graph = [[] for _ in range(N + 1)]
    for u, u_p in enumerate(map(int, input().split()), start=1):
        graph[u].append(u_p)
        graph[u_p].append(u)
    M = int(input())
    START, END = 0, 1
    NTH = 0
    ON, OFF = 1, -1
    root, none = 1, 0
    query_ranges = [[0] * 2 for _ in range(N + 1)]
    dfs(graph, root, none, query_ranges)
    h = math.ceil(math.log2(N + 1))
    tree_size = 2**(h + 1)
    tree = [0] * tree_size
    lazy = [0] * tree_size
    start, end = 0, N - 1
    update_range(*query_ranges[root], ON, tree, lazy, root, start, end)
    answers = []
    for _ in range(M):
        operator, i = map(int, input().split())
        left, right = query_ranges[i]
        if left > right:
            if operator == 3:
                answers.append(0)
            continue
        if operator == 1:
            update_range(left, right, ON, tree, lazy, root, start, end)
        elif operator == 2:
            update_range(left, right, OFF, tree, lazy, root, start, end)
        else:
            count = range_query(left, right, tree, lazy, root, start, end)
            answers.append(count)
    print("\n".join(map(str, answers)))


solution()

# input
# 3
# 0 1 2
# 8
# 3 1
# 2 1
# 3 1
# 1 1
# 3 1
# 2 2
# 3 1
# 3 2

 3
 0 1 2
 8
 3 1
 2 1
 3 1
 1 1
 3 1
 2 2
 3 1
 3 2


2
0
2
1
0


## 회사 문화 4

- 문제 출처: [백준 14288번](https://www.acmicpc.net/problem/14288)

`-` [회사 문화 2](https://www.acmicpc.net/problem/14268) + [회사 문화 3](https://www.acmicpc.net/problem/14287)

`-` 세그먼트 트리를 각각 만들어서 관리하자

In [8]:
import math
import sys

sys.setrecursionlimit(10**5 + 2)


def dfs(graph, node, query_ranges, indices):
    global NTH
    NTH += 1
    query_ranges[node][IN] = NTH
    indices[node] = NTH
    for child in graph[node]:
        dfs(graph, child, query_ranges, indices)
    query_ranges[node][OUT] = NTH


def update_lazy(tree, lazy, node, start, end):
    if lazy[node] == 0:
        return
    tree[node] += lazy[node]
    if start != end:
        lazy[2 * node] += lazy[node]
        lazy[2 * node + 1] += lazy[node]
    lazy[node] = 0


def update_range(left, right, value, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if left > end or right < start:
        return
    if left <= start and end <= right:
        lazy[node] += value
        update_lazy(tree, lazy, node, start, end)
        return
    mid = (start + end) // 2
    update_range(left, right, value, tree, lazy, 2 * node, start, mid)
    update_range(left, right, value, tree, lazy, 2 * node + 1, mid + 1, end)
    tree[node] = tree[2 * node] + tree[2 * node + 1]


def point_query(index, tree, lazy, node, start, end):
    update_lazy(tree, lazy, node, start, end)
    if index < start or index > end:
        return 0
    if start == end:
        return tree[node]
    mid = (start + end) // 2
    left_value = point_query(index, tree, lazy, 2 * node, start, mid)
    right_value = point_query(index, tree, lazy, 2 * node + 1, mid + 1, end)
    return left_value + right_value


def update_point(index, value, tree, node, start, end):
    if index < start or index > end:
        return
    if start == end:
        tree[node] += value
        return
    mid = (start + end) // 2
    update_point(index, value, tree, 2 * node, start, mid)
    update_point(index, value, tree, 2 * node + 1, mid + 1, end)
    tree[node] = tree[2 * node] + tree[2 * node + 1]


def range_sum_query(left, right, tree, node, start, end):
    if left > end or right < start:
        return 0
    if left <= start and end <= right:
        return tree[node]
    mid = (start + end) // 2
    left_sum = range_sum_query(left, right, tree, 2 * node, start, mid)
    right_sum = range_sum_query(left, right, tree, 2 * node + 1, mid + 1, end)
    return left_sum + right_sum


def solution():
    global IN, OUT, NTH
    n, m = map(int, input().split())
    graph = [[] for _ in range(n + 1)]
    for u, u_p in enumerate(map(int, input().split()), start=1):
        if u_p == -1:
            continue
        graph[u_p].append(u)
    UP, DOWN = 0, 1
    IN, OUT = 0, 1
    NTH = -1
    root = 1
    query_ranges = [[0] * 2 for _ in range(n + 1)]
    indices = [0] * (n + 1)
    dfs(graph, root, query_ranges, indices)
    h = math.ceil(math.log2(n))
    tree_size = 2**(h + 1)
    tree = [0] * tree_size
    tree_reverse = [0] * tree_size
    lazy = [0] * tree_size
    start, end = 0, n - 1
    direction = DOWN
    answers = []
    for _ in range(m):
        query = list(map(int, input().split()))
        operator = query[0]
        if operator == 1:
            _, i, w = query
            if direction == UP:
                update_point(indices[i], w, tree_reverse, root, start, end)
                continue
            left, right = query_ranges[i]
            update_range(left, right, w, tree, lazy, root, start, end)
        elif operator == 2:
            _, i = query
            value = point_query(indices[i], tree, lazy, root, start, end)
            left, right = query_ranges[i]
            range_sum = range_sum_query(left, right, tree_reverse, root, start, end)
            answer = value + range_sum
            answers.append(answer)
        else:
            direction = 1 - direction
    print("\n".join(map(str, answers)))


solution()

# input
# 5 8
# -1 1 2 3 4
# 1 2 2
# 3
# 1 3 4
# 3
# 1 4 6
# 2 5
# 2 3
# 2 1

 5 8
 -1 1 2 3 4
 1 2 2
 3
 1 3 4
 3
 1 4 6
 2 5
 2 3
 2 1


8
6
4
